### New and Improved Loss Function

In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
import kornia.color as convert
# For our model
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
# For utilities
import os, shutil, time

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
use_gpu = torch.cuda.is_available()

In [3]:
class ColorizationNet(nn.Module):
  def __init__(self, input_size=128):
    super(ColorizationNet, self).__init__()
    MIDLEVEL_FEATURE_SIZE = 128

    ## First half: ResNet
    resnet = models.resnet18(num_classes=365) 
    # Change first conv layer to accept single-channel (grayscale) input
    #resnet.conv1.weight = nn.Parameter(resnet.conv1.weight.sum(dim=1).unsqueeze(1)) 
    # Extract midlevel features from ResNet-gray
    self.midlevel_resnet = nn.Sequential(*list(resnet.children())[0:6])

    ## Second half: Upsampling
    self.upsample = nn.Sequential(     
      nn.Conv2d(MIDLEVEL_FEATURE_SIZE, 128, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Conv2d(32, 2, kernel_size=3, stride=1, padding=1),
      nn.Upsample(scale_factor=2)
    )

  def forward(self, input):

    # Pass input through ResNet-gray to extract features
    midlevel_features = self.midlevel_resnet(input)

    # Upsample to get colors
    output = self.upsample(midlevel_features)
    return output

In [4]:
model = ColorizationNet()

In [5]:
criterion = LearnedPerceptualImagePatchSimilarity(net_type='vgg')

/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.0)

In [7]:
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

In [8]:
class GrayscaleImageFolder(datasets.ImageFolder):
  '''Custom images folder, which converts images to grayscale before loading'''
  def __getitem__(self, index):
    path, target = self.imgs[index]
    img = self.loader(path)
    if self.transform is not None:
        img_original = self.transform(img)

        img_rgb = to_tensor(img_original)

        img_lab = convert.rgb_to_lab(img_rgb)

        img_ab = img_lab[1:3,:,:]

        img_gray = img_lab[0,:,:]

        img_gray = torch.stack([img_gray, torch.zeros(img_gray.shape), torch.zeros(img_gray.shape)])


    #   img_original = self.transform(img)
    #   img_original = np.asarray(img_original)
    #   img_lab = rgb2lab(img_original)
    #   img_lab = (img_lab + 128) / 255
    #   img_ab = img_lab[:, :, 1:3]
    #   img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
    #   img_original = rgb2gray(img_original)
    #   img_original = torch.from_numpy(img_original).unsqueeze(0).float()
    # if self.target_transform is not None:
    #   target = self.target_transform(target)
    return img_gray, img_rgb, target

In [9]:
# Training
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip()])
train_imagefolder = GrayscaleImageFolder('DS5660_project/sport_photos/train', train_transforms)
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=64, shuffle=True)

# Validation 
val_transforms = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224)])
val_imagefolder = GrayscaleImageFolder('DS5660_project/sport_photos/valid' , val_transforms)
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=64, shuffle=False)


In [10]:
class AverageMeter(object):
  '''A handy class from the PyTorch ImageNet tutorial''' 
  def __init__(self):
    self.reset()
  def reset(self):
    self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
  '''Show/save rgb image from grayscale and ab channels
     Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
  plt.clf() # clear matplotlib 
  color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
  color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
  color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
  color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
  color_image = lab2rgb(color_image.astype(np.float64))
  grayscale_input = grayscale_input.squeeze().numpy()
  if save_path is not None and save_name is not None: 
    plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
    plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [11]:
def validate(val_loader, model, criterion, save_images, epoch):
  model.eval()

  # Prepare value counters and timers
  batch_time, data_time, losses = AverageMeter(), AverageMeter(), AverageMeter()

  end = time.time()
  already_saved_images = False
  for i, (input_gray, input_ab, target) in enumerate(val_loader):
    data_time.update(time.time() - end)

    # Use GPU
    if use_gpu: input_gray, input_ab, target = input_gray.cuda(), input_ab.cuda(), target.cuda()

    # Run model and record loss
    output_ab = model(input_gray)
    greys = input_gray[:,0,:,:]
    greys = greys.unsqueeze(1)
    output_a = output_ab[:,0,:,:]
    output_b = output_ab[:,1,:,:]
    output_a = output_a.unsqueeze(1)
    output_b = output_b.unsqueeze(1)
    output_ab = torch.cat([greys,output_a,output_b],dim=1)
    output_ab = convert.lab_to_rgb(output_ab)
    loss = criterion(output_ab, input_ab)
    losses.update(loss.item(), input_gray.size(0))

    # Save images to file
    if save_images and not already_saved_images:
      already_saved_images = True
      for j in range(min(len(output_ab), 10)): # save at most 5 images
        save_path = {'grayscale': 'outputs/gray/', 'colorized': 'outputs/color/'}
        save_name = 'img-{}-epoch-{}.jpg'.format(i * val_loader.batch_size + j, epoch)
        if save_path is not None and save_name is not None:
            save_color = to_pil(output_ab[j].detach().cpu())
            save_gray = to_pil(input_gray[j].cpu())
            save_color.save(save_path['colorized']+save_name)
            save_gray.save(save_path['grayscale']+save_name)
        #to_rgb(input_gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    # Record time to do forward passes and save images
    batch_time.update(time.time() - end)
    end = time.time()

    # Print model accuracy -- in the code below, val refers to both value and validation
    if i % 25 == 0:
      print('Validate: [{0}/{1}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
             i, len(val_loader), batch_time=batch_time, loss=losses))

  print('Finished validation.')
  return losses.avg

In [12]:
def train(train_loader, model, criterion, optimizer, epoch):
  print('Starting training epoch {}'.format(epoch))
  model.train()
  
  # Prepare value counters and timers
  batch_time, data_time, losses = AverageMeter(), AverageMeter(), AverageMeter()

  end = time.time()
  for i, (input_gray, input_ab, target) in enumerate(train_loader):
    
    # Use GPU if available
    if use_gpu: input_gray, input_ab, target = input_gray.cuda(), input_ab.cuda(), target.cuda()

    # Record time to load data (above)
    data_time.update(time.time() - end)

    # Run forward pass
    output_ab = model(input_gray)
    greys = input_gray[:,0,:,:]
    greys = greys.unsqueeze(1)
    output_a = output_ab[:,0,:,:]
    output_b = output_ab[:,1,:,:]
    output_a = output_a.unsqueeze(1)
    output_b = output_b.unsqueeze(1)
    output_ab = torch.cat([greys,output_a,output_b],dim=1)
    output_ab = convert.lab_to_rgb(output_ab)
    loss = criterion(output_ab, input_ab) 
    losses.update(loss.item(), input_gray.size(0))

    # Compute gradient and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Record time to do forward and backward passes
    batch_time.update(time.time() - end)
    end = time.time()

    # Print model accuracy -- in the code below, val refers to value, not validation
    if i % 25 == 0:
      print('Epoch: [{0}][{1}/{2}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
              epoch, i, len(train_loader), batch_time=batch_time,
             data_time=data_time, loss=losses)) 

  print('Finished training epoch {}'.format(epoch))

In [13]:
if use_gpu: 
  criterion = criterion.cuda()
  model = model.cuda()

In [14]:
# Make folders and set parameters
os.makedirs('outputs/color', exist_ok=True)
os.makedirs('outputs/gray', exist_ok=True)
os.makedirs('new_checkpoints', exist_ok=True)
save_images = True
best_losses = 1e10
epochs = 186

In [15]:
# Train model
for epoch in range(epochs):
  # Train for one epoch, then validate
  train(train_loader, model, criterion, optimizer, epoch)
  with torch.no_grad():
    losses = validate(val_loader, model, criterion, save_images, epoch)
  # Save checkpoint and replace old best model if current model is better
  if losses < best_losses:
    best_losses = losses
    torch.save(model.state_dict(), 'new_checkpoints/model-epoch-{}-losses-{:.3f}.pth'.format(epoch+1,losses))

Starting training epoch 0
Epoch: [0][0/213]	Time 2.419 (2.419)	Data 0.355 (0.355)	Loss 0.1767 (0.1767)	
Epoch: [0][25/213]	Time 0.330 (0.521)	Data 0.218 (0.333)	Loss 0.1520 (0.1631)	
Epoch: [0][50/213]	Time 0.367 (0.444)	Data 0.254 (0.292)	Loss 0.1476 (0.1638)	
Epoch: [0][75/213]	Time 0.364 (0.421)	Data 0.253 (0.282)	Loss 0.1675 (0.1635)	
Epoch: [0][100/213]	Time 0.351 (0.404)	Data 0.239 (0.272)	Loss 0.1603 (0.1627)	
Epoch: [0][125/213]	Time 0.358 (0.401)	Data 0.245 (0.272)	Loss 0.1582 (0.1623)	
Epoch: [0][150/213]	Time 0.364 (0.394)	Data 0.251 (0.268)	Loss 0.1586 (0.1625)	
Epoch: [0][175/213]	Time 0.350 (0.393)	Data 0.237 (0.269)	Loss 0.1520 (0.1617)	
Epoch: [0][200/213]	Time 0.341 (0.390)	Data 0.228 (0.267)	Loss 0.1568 (0.1615)	
Finished training epoch 0
Validate: [0/8]	Time 0.421 (0.421)	Loss 0.1793 (0.1793)	
Finished validation.
Starting training epoch 1
Epoch: [1][0/213]	Time 0.374 (0.374)	Data 0.260 (0.260)	Loss 0.1628 (0.1628)	
Epoch: [1][25/213]	Time 0.375 (0.380)	Data 0.261 (0